In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import sys 
import iris
import re
from concurrent.futures import ThreadPoolExecutor

sys.path.insert(1, '../FindingEvents/')
from Identify_Events_Functions import *
from Prepare_Data_Functions import *
from Get_Events_Functions import *

gauge_nums = range(445,1294)
em ='bc009'
time_period='Present'
yrs_range ='2002_2020'
sample_cube_yr = 2006
yr_1, yr_2 = 2001, 2020

In [ ]:
full_yr_cubes_dict = {}
for yr in range(yr_1, yr_2):
    # Load data for that year
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}/WholeYear/cube_{yr}.pkl"
    full_yr_cubes_dict[yr]= load_cube_from_picklefile(pickle_file_filepath)

for em in [em]:
    for gauge_num in gauge_nums:
        if gauge_num not in [444, 827, 888]:
            
            missing_files = False
            if em == 'nimrod':
                indy_events_fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/2km_filtered_100/{gauge_num}/WholeYear/"
            else:
                indy_events_fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/"

            base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents_precedingtime/UKCP18_30mins/{em}/{gauge_num}/WholeYear"
            # Create the directory if it doesnt exist
            if not os.path.isdir(base_dir):
                os.makedirs(base_dir)                

            # Check if we are missing any of the files, and if so, change the flag to True
            if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                missing_files = True           
                
            if missing_files == True:    
                print("Files missing")
                # Get Tb0 values at each gauge
                tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
                sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{sample_cube_yr}01.nc')[0][1,:,:]
                Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)


                print(f"Processing gauge {gauge_num}")
    #             if gauge_num % 100 == 0:
    #                 print(f"Processing gauge {gauge_num}")


                files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
                files = np.sort(files)

                for file in files:
                    fp = indy_events_fp +  f"{file}"
                    # Get the event
                    this_event = read_event(gauge_num, fp)
                    # Find the year
                    match = re.search(r'20\d{2}', fp)
                    yr = match.group(0)
                    # Get data
                    full_year_cube =  full_yr_cubes_dict[int(yr)]
                    # Extract data for the gauge location
                    one_location_cube = full_year_cube[:, idx_2d[0], idx_2d[1]]

                    # Get the 100 timeslices before this event
                    end_index = this_event['Unnamed: 0'][0]
                    if end_index<100:
                        start_index = end_index-end_index
                    else:
                        start_index=end_index-100

                    if end_index == 0:
                        preceding_times_df = pd.DataFrame()
                        print(f'Start index was 0 for {file}')
                    else:

                        # Assuming one_location_cube is your Iris cube and start_index is defined
                        time_coord = one_location_cube.coord('time')
                        time_unit = time_coord.units

                        time_datetimes = time_unit.num2date(one_location_cube[start_index:end_index].coord('time').points)

                        # Create a DataFrame with the datetime times
                        preceding_times_df = pd.DataFrame({'times': time_datetimes,
                            'precip': one_location_cube[start_index:end_index].data})

                    # Display the DataFrame
                    preceding_times_df.to_csv(f"{base_dir}/{file}", index=False)

Files missing
Processing gauge 488
Files missing
Processing gauge 489
Files missing
Processing gauge 490
Files missing
Processing gauge 491
Files missing
Processing gauge 492
Files missing
Processing gauge 493
Files missing
Processing gauge 494
Files missing
Processing gauge 495
Files missing
Processing gauge 496
Files missing
Processing gauge 497
Files missing
Processing gauge 498
Files missing
Processing gauge 499
Files missing
Processing gauge 500
Files missing
Processing gauge 501
Files missing
Processing gauge 502
Files missing
Processing gauge 503
